In [1]:
import pandas as pd
from  sklearn.linear_model import SGDClassifier
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [8]:
from lightgbm import LGBMClassifier

In [23]:
from sklearn.model_selection import KFold

### Загружаем данные в виде pandas.DataFrame и делим на test и train

In [2]:
df = pd.read_csv('test_result.csv', delimiter=',')
df.shape

(121, 157)

In [3]:
train = df[:90]
test  = df[90:]

In [8]:
bad = []
# Проверяем на наличие NULL-ов
for a in test:
    if test[a].isnull().sum() != 0:
        print(a, test[a].isnull().sum())
        bad.append(a)

moments_normalized-0-0 31
moments_normalized-0-1 31
moments_normalized-1-0 31
weighted_moments_normalized-0-0 31
weighted_moments_normalized-0-1 31
weighted_moments_normalized-1-0 31


In [4]:
# Чтоб не париться заполняем NULL-ы нулями
train = train.fillna(0)
test = test.fillna(0)

In [12]:
test

,Unnamed: 0,area,bbox-0,bbox-1,bbox-2,bbox-3,bbox_area,centroid-0,centroid-1,convex_area,...,weighted_moments_normalized-2-2,weighted_moments_normalized-2-3,weighted_moments_normalized-3-0,weighted_moments_normalized-3-1,weighted_moments_normalized-3-2,weighted_moments_normalized-3-3,img_index,corner_points,target,predict
90,25,809,639,538,703,602,4096,670.802225,569.463535,2443,...,1.008847e-06,2.206446e-09,-1.457406e-06,2.864197e-07,4.810418e-10,-1.661298e-10,2,[[ 4 30]\n [44 7]\n [44 11]\n [45 4]\n [46 1...,квадрат,квадрат
91,26,742,674,379,730,434,3080,701.513477,406.198113,2004,...,9.510458e-07,5.075229e-10,1.079370e-06,-3.657201e-07,9.811192e-10,4.506534e-11,2,[[ 3 40]\n [42 12]\n [42 52]\n [42 56]\n [43 6...,квадрат,квадрат
92,27,481,732,635,777,679,1980,757.983368,659.528067,1166,...,9.813716e-07,1.780944e-08,-2.045544e-05,-1.340337e-07,7.181975e-09,3.635894e-10,2,[[ 4 35]\n [42 19]\n [42 23]\n [42 27]\n [42 3...,треугольник,треугольник
93,28,576,739,266,792,299,1749,765.286458,282.250000,1268,...,8.071897e-07,2.071615e-09,-4.999538e-07,-7.186459e-07,-2.551195e-09,-8.687535e-10,2,[[ 3 20]\n [41 34]\n [42 5]\n [42 9]\n [42 1...,прямоугольник,прямоугольник
94,29,656,746,780,794,835,2640,768.754573,807.382622,1961,...,1.520146e-06,-2.380416e-10,2.940072e-06,-5.937416e-07,5.792743e-09,-6.232861e-10,2,[[ 4 46]\n [40 16]\n [40 42]\n [40 46]\n [40 5...,прямоугольник,прямоугольник
95,30,932,776,286,822,364,3588,800.549356,326.415236,2421,...,8.952246e-07,2.235272e-08,-7.303151e-06,-5.050576e-07,-9.283563e-09,-9.877971e-10,2,[[ 2 6]\n [34 17]\n [34 31]\n [34 35]\n [34 3...,mixed,mixed
96,31,863,783,443,845,498,3410,815.205098,471.582851,2081,...,6.469590e-07,2.031824e-09,-5.765936e-06,7.046299e-07,3.332432e-09,7.136234e-10,2,[[ 4 27]\n [51 23]\n [51 27]\n [51 49]\n [51 5...,mixed,mixed
97,32,842,803,680,837,738,1972,819.142518,705.077197,1610,...,4.766665e-07,3.132360e-09,8.698241e-07,-1.070361e-08,-3.983200e-09,2.482239e-11,2,[[ 4 6]\n [29 23]\n [29 27]\n [29 31]\n [29 3...,mixed,mixed
98,33,503,820,749,859,788,1521,838.465209,767.624254,1207,...,6.706731e-07,-3.660185e-10,1.400963e-06,-9.296211e-09,2.364921e-09,4.479397e-13,2,[[ 4 18]\n [33 7]\n [32 13]\n [31 44]\n [38 2...,круг,круг
99,0,376,91,537,138,583,2162,114.138298,559.587766,1266,...,1.137065e-06,-6.222525e-11,-5.796967e-07,2.955128e-07,2.220741e-10,6.949889e-10,3,[[ 3 27]\n [38 44]\n [39 40]\n [44 32]\n [34 5...,прямоугольник,квадрат


In [13]:
test.to_csv("example_predict.csv")

### Создаём и обучаем простые модели

In [6]:
features = ['bbox_area', 'convex_area', 'eccentricity', 'equivalent_diameter', 'filled_area', 'major_axis_length', 'minor_axis_length','perimeter','perimeter_crofton','solidity']

In [42]:
# KNN Classifier
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(train[features], train['target'])

predict = clf.predict(test[features])

In [9]:
# DecisionTree
clf = DecisionTreeClassifier()
clf.fit(train[features], train['target'])

predict = clf.predict(test[features])

In [12]:
# Random Forest
clf = RandomForestClassifier(n_estimators = 10)
clf.fit(train[features], train['target'])

predict = clf.predict(test[features])

In [9]:
# Boosting
model = LGBMClassifier(n_estimators = 500, max_depth = 3)
model.fit(train[features], train['target'])

predict = model.predict(test[features])

In [10]:
test['predict'] = predict

In [11]:
# Считаю процент верных предположений
def accuracy(test):
    i = 0
    for index, row in test.iterrows():
        if row['target'] == row['predict']:
            i = i+1

    perc = i/test.shape[0]
    return perc

per = accuracy(test)
print(per)

0.7419354838709677


In [53]:
# Считаю процент верных предположений
y_test = list(test['target'])
predict = list(test['predict'])
def acc(y_test, predict):
    i = 0
    for k in range(len(y_test)):
        if y_test[k] == predict[k]:
            i = i+1
    perc = i/len(y_test)
    return perc

per = acc(y_test, predict)
print(per)

0.7419354838709677


### Кроссвалидация

In [62]:
kf = KFold(n_splits=5)
kf.get_n_splits(train)

5

In [63]:
model = LGBMClassifier(n_estimators = 500, max_depth = 3)

acc_list = []

for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    X_train, X_test = train.loc[train_index, features], train.loc[test_index, features ]
    y_train, y_test = train.loc[train_index, 'target'], train.loc[test_index, 'target']

    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    accur = acc(list(y_test), list(predict))
    acc_list.append(accur)
    print(i, accur)
    
print("mean rmse for 5-fold: {}".format(str(np.mean(acc_list))))
print("std roc_auc for 5-fold: {}".format(str(np.std(acc_list))))

0 0.6666666666666666
1 0.6111111111111112
2 0.6111111111111112
3 0.8333333333333334
4 0.4444444444444444


NameError: name 'np' is not defined